# 546 (Optimization for Learning and Control) hw0

You are welcome (and encouraged) to work with others, but each individual must submit their own writeup.

You are welcome to use analytical and numerical computational tools; if you do, include the **commented** sourcecode in your writeup (e.g. the .ipynb file).

You are welcome to consult research articles and other materials; if you do, include a full citation in your writeup (e.g. the .ipynb file) and upload a .pdf of the article to Canvas alongside your homework submission.

## linear quadratic regulator

Consider the discrete-time linear quadratic regulator problem (DT-LQR)

$$\min_u c_\tau (x,u) \ \text{s.t.} \ x_{s+1} = A_s x_s + B_s u_s,$$
$$c_\tau(x,u) = \frac{1}{2} x_t^T P_t x_t + \frac{1}{2}\sum_{s=\tau}^{t-1} x_s^T Q_s x_s + u_s^T R_s u_s$$

where $P_t$, $\{Q_s\}_{s=\tau}^{t-1}$, $\{R_s\}_{s=\tau}^{t-1}$ are given symmetric positive-definite matrices.

In this problem, you'll apply Bellman's dynamic programming principle to determine the optimal control, then verify this control is the same one obtained by a descent algorithm.

a. Determine the optimal control and value at time $\tau = t$ (i.e. minimize $c_t$ with respect to $u_t$, and let $v^*_t = c_t(x^*,u^*)$ denote the optimal value that can be obtained at time $t$).

For $\tau = t$ the optimal control can be any thing, since optimal control $u^*_t$ determine the system state at $t+1$ which we don't care about. And the cost is the current system state value $v_t^* = \frac{1}{2} x_t^T P_t x_t$

b. Determine the optimal control and value at time $\tau = t-1$ (i.e. minimize $c_{t-1}$ with respect to $u_{t-1}$, and let $v^*_{t-1} = c_{t-1}(x^*,u^*)$ denote the optimal value that can be obtained at time $t-1$).

At time $\tau = t-1$, cost can be found:
$$J_{t-1} = \frac{1}{2} x^T_t P_t x_t + \frac{1}{2}x^T_{t-1} Q_{t-1} x_{t-1} + \frac{1}{2}u^T_{t-1} R_{t-1} u_{t-1}$$
with system constrain:
$$x_t = A_{t-1} x_{t-1} + B_{t-1}u_{t-1}$$

Applying nonlinear programming:

$D_{u_{t-1}} c_{t-1} = u^T_{t-1} R_{t-1} + x^T_t P(D_{u_{t-1}} x_t)$

$u^*_{t-1} = -(B^T_{t-1} P_t B_{t-1} + R_{t-1})^{-1} B_{t-1}^T P_t A_{t-1} x_{t-1}$

where the second derivative of the cost function needs to be positive:

$D_{u_{t-1}}^2 c_{t-1} = R_{t-1} +B_{t-1}^T P_t B_{t-1} > 0$

Then for cost in $\tau = t-1$:
$$v_{t-1}^* = \frac{1}{2} x_{t-1}^T P_{t-1} x_{t-1}$$

c. Note that the optimal control is linear in state and the optimal value is quadratic in state (i.e. determine $K_{t-1}$ and $P_{t-1}$ such that $u_{t-1} = K_{t-1} x_{t-1}$ and $v^*_{t-1} = c_{t-1}(x^*,u^*) = x_{t-1}^T P_{t-1} x_{t-1}$).

As we've calculated the optimal control for $\tau = t-1$ above:
$$K_{t-1} = -(B_{t-1}^T P_{t} B_{t-1} + R_{t-1})^{-1} B_{t-1}^T P_{t-1} A_{t-1}$$
$$P_{t-1} = (A_{t-1} - B_{t-1} K_{t-1})^T P{t} (A_{t-1} - B_{t-1} K_{t-1}) + K_{t-1}^T R_{t-1} K_{t-1} + Q_{t-1}$$

d. Use the substitution in (c.) to generalize the calculation in (b.) to derive a recursive formula to compute the optimal control and value at any time $s\in [\tau,t]$.

For each time step $s \in [\tau,t]$ in a trajectory:

$$u_s^* = -K_s x_s$$
$$J_s^* = \frac{1}{2} x_s^T P_s x_s$$
$$-K_s = (B_s^T P_{s+1} B_s + R_s)^{-1} B_s^T P_{s+1} A_s$$
$$P_s = (A_s - B_s K_s)^T P_{s+1} (A_s - B_s K_s) + K_s^T R_s K_s + Q_s$$

e. Implement the recursion in (d.) numerically for the following system, which is a discretized linear spring-mass-damper (let $\kappa,\beta,q,r,\Delta > 0$ be parameters that you can easily vary in your implementation):

$$A^\Delta = I + \Delta \left[\begin{array}{cc} 0 & 1 \\ -\kappa & -\beta \end{array}\right],\ B^\Delta = \Delta \left[\begin{array}{c} 0 \\ 1 \end{array}\right],\ P_t = 0,\ Q^\Delta_s = q \cdot \Delta \cdot I,\ R^\Delta_s = r \cdot \Delta \cdot I.$$

In [9]:
import numpy as np
import math
import numpy.linalg as la
import scipy.linalg as sla
import matplotlib.pyplot as plt

In [15]:
tri, k, b, q, r = 0.1, 1.0, 1.0, 1.0, 1.0

'''this function is modified from Sam Burden's EE548 lecture example '''
def LQR(A,B,Q,R,Pt,tf, dt=1e-4):
    t = 0.0
    K = []
    P = []
    while t < tf:
        Pi = sla.solve_continuous_are(A(tri, k, b, q, r),B(tri, k, b, q, r),\
                                      Q(tri, k, b, q, r),R(tri, k, b, q, r));
        K.append(np.dot(la.inv(R(tri, k, b, q, r)), np.dot(B(tri, k, b, q, r).T,Pi)));
        P.append(Pi);
        t += dt;
    K.append(np.dot(la.inv(R(tri, k, b, q, r)), np.dot(B(tri, k, b, q, r).T,Pt)))
    P.append(Pt)
    return np.asarray(K), np.asarray(P)

In [18]:
A = lambda tri, k, b, q, r : np.identity(2) + np.dot(tri, np.array([[0.0,1.0],[-k,-b]]))
B = lambda tri, k, b, q, r : np.dot(tri, np.array([[0],[1]]))
Q = lambda tri, k, b, q, r : np.dot(q*tri, np.identity(2))
R = lambda tri, k, b, q, r : np.dot(r*tri, np.identity(1))
P = np.zeros((2,2));
K,P = LQR(A,B,Q,R,P,10,0.1)

f. Verify your implementation in (e.) by performing gradient descent to approximate the optimal input on time horizon $t = 10\cdot\Delta$ and $t = 100\cdot\Delta$ for the following initial conditions with $\kappa,\beta,q,r = 1$, $\Delta = 0.1$:

$$x_1 = (1,0),\ x_2 = (0,1).$$

g. Investigate how the optimal controls vary with respect to the ratio $q/r$ (i.e. let $q/r$ range from $0.1$ to $10$, compute the optimal control input, describe the trend you observe, and provide a visualization that clearly illustrates this trend).

h. Compare the results from (e.)/(f.) with the results obtained from the solution to the continuous-time linear quadratic regulator (CT-LQR):

$$A = \frac{1}{\Delta} (A^\Delta - I),\ B = \frac{1}{\Delta} B^\Delta,\ Q = \frac{1}{\Delta} Q^\Delta,\ R = \frac{1}{\Delta} R^\Delta,$$

$$u^*_s = - K_s x_s,\ K_s = R^{-1} B^T P_s,$$

$$v^*_s = \frac{1}{2} x_s^T P_s x_s,\ \dot{P_s} = -(A^T P_s + P_s A - P_s B R^{-1} B^T P_s + Q).$$

Investigate how the comparison varies with respect to $\Delta$ (i.e. let $\Delta$ range from $\sim 10^0$ to $\sim 10^{-8}$ and plot the norm of the difference between the optimal inputs and values as a function of $\Delta$). 

**BONUS** show that the solution to (CT-LQR) can be obtained by taking the limit of the (DT-LQR) recursion from (d.)/(e.) as $\Delta \rightarrow 0$.

## application

a. Specify an optimal control problem for a system of your choosing (e.g. a system from the paper you plan to present or the project you plan to complete for the class).  Be precise:  provide the mathematical definitions for all symbols.  Also, be clear:  provide plain language explanations for what each symbol corresponds to in the application.

b. Discuss what methods have been applied in the literature to solve or approximate solutions to the optimal control problem from (a.), and what (if anything) is known about the solution.

c. Attempt to apply a technique for optimal control to the problem from (a.).  You can try one of the methods discussed in class (e.g. directly apply the dynamic programming principle, or apply a numerical descent algorithm), one of the methods discussed in (b.), or another method of your choosing.  